In [8]:
import geemap
import ee

In [3]:
# Authenticate to Earth Engine
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=MWFF8vNOoeuEiVAyIkVcsBR_4dhBYdnC2j6ry6RRIQI&tc=E4PZc_CsZMQ4yPVZFTMZrJCgbG_7PwUUhi6JBIGXpDE&cc=RMoA_NNOICHQkhq-OuCjvfcReCgNmRJifc1FTIG5GHo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXn609MiwRVo5VV6G7JkAIIQTQtsWqONqmUm4Bt4KJVk6k0hPknHtU8

Successfully saved authorization token.


In [4]:
# Construct a point from coordinates.
geometry = ee.Geometry.Point([-47.7330, -3.8850])

In [5]:
# Define a buffer around the point
bufferDistance = 10000
pointBuffer = geometry.buffer(bufferDistance)

In [10]:
# Choose dates for the pre-fire and post-fire period
preFireStartDate = ee.Date.fromYMD(2019, 6, 1)
preFireEndDate = ee.Date.fromYMD(2019, 8, 31)

postFireStartDate = ee.Date.fromYMD(2020, 6, 1)
postFireEndDate = ee.Date.fromYMD(2020, 8, 31)

In [11]:
# Function to mask clouds in Sentinel-2 data
def mask_s2_clouds(image):
    qa60 = image.select(['QA60'])
    cloud_mask = qa60.bitwiseAnd(1 << 10).eq(0).And(qa60.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(cloud_mask)

In [27]:
    # Filter Sentinel-2 data for pre- and post-fire periods
    pre_fire_s2 = (
        ee.ImageCollection('COPERNICUS/S2')
        .filterBounds(pointBuffer)
        .filterDate(preFireStartDate, preFireEndDate)
        .map(mask_s2_clouds)
    )

    post_fire_s2 = (
        ee.ImageCollection('COPERNICUS/S2')
        .filterBounds(pointBuffer)
        .filterDate(postFireStartDate, postFireEndDate)
        .map(mask_s2_clouds)
    )

In [28]:
    # Create median images for pre- and post-fire periods
    pre_fire_s2_median = pre_fire_s2.median().clip(pointBuffer)
    post_fire_s2_median = post_fire_s2.median().clip(pointBuffer)

In [29]:
    # Calculate NBR for pre- and post-fire
    pre_fire_nbr = pre_fire_s2_median.normalizedDifference(['B8', 'B4']).rename('NBR').toFloat()
    post_fire_nbr = post_fire_s2_median.normalizedDifference(['B8', 'B4']).rename('NBR').toFloat()

    # Calculate dNBR (pre-fire NBR - post-fire NBR)
    d_nbr = pre_fire_nbr.subtract(post_fire_nbr).rename('dNBR').toFloat()

In [33]:
 # Define severity classes based on the provided dNBR ranges
severity = (
    d_nbr.where(d_nbr.lt(-0.1), 0)
    .where(d_nbr.gte(-0.1).And(d_nbr.lte(0.1)), 1)
    .where(d_nbr.gt(0.1).And(d_nbr.lte(0.27)), 2)
    .where(d_nbr.gt(0.27).And(d_nbr.lte(0.66)), 3)
    .where(d_nbr.gt(0.66), 4)
)

In [46]:
# Display the burn severity index on the map
Map = geemap.Map()
Map.centerObject(pointBuffer, 15)

# Add the pre-fire RGB layer to the map
Map.addLayer(pre_fire_s2_median, vis_params, 'Pre-Fire RGB')

# Add the post-fire RGB layer to the map
Map.addLayer(post_fire_s2_median, vis_params, 'Post-Fire RGB')

# Display the dNBR layer on the map
Map.addLayer(d_nbr, d_nbr_vis_params, 'dNBR')

# Add the pointBuffer to the map
Map.addLayer(pointBuffer, {'color': 'FF0000'}, 'Buffer')

# Set visualization parameters for the severity index
severity_vis_params = {'min': 0, 'max': 4, 'palette': ['00FF00', 'FFFF00', 'FFA500', 'FF4500', 'FF0000']}
Map.addLayer(severity, severity_vis_params, 'Burn Severity Index')

# Create a legend
legend_title = 'Burn Severity Legend'
legend_labels = ['Enhanced Regrowth', 'Unburned', 'Low Severity', 'Moderate Severity', 'High Severity']
legend_colors = ['00FF00', 'FFFF00', 'FFA500', 'FF4500', 'FF0000']

# Add legend to the map
Map.add_legend(title=legend_title, labels=legend_labels, colors=legend_colors)

# Display the map
Map

Map(center=[-3.8849897411263044, -47.73299971485229], controls=(WidgetControl(options=['position', 'transparen…